In [20]:
import torch
torch.cuda.is_available()
print(torch.zeros(1).cuda())

import torch_geometric
import torch_geometric.nn as pyg_nn
from torch_sparse import SparseTensor, matmul
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
print(torch_geometric.__version__)

import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType, OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

import networkx as nx
from deepsnap.hetero_graph import HeteroGraph
import copy
from copy import deepcopy
import pickle

import deepsnap
from deepsnap.graph import Graph
from deepsnap.batch import Batch
from deepsnap.dataset import GraphDataset
from deepsnap.hetero_gnn import forward_op
from deepsnap.hetero_graph import HeteroGraph
from sklearn.metrics import f1_score, roc_auc_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
from pathlib import Path as Data_Path
from os import listdir
from os.path import isfile, join
from itertools import combinations
from tqdm.notebook import tqdm

# import graph_tool.all as gt
import graph_tool.all as gt
import json
print("graph-tool version: {}".format(gt.__version__.split(' ')[0]))

tensor([0.], device='cuda:0')
2.1.0
graph-tool version: 2.46


In [21]:
def json2graph(g, json_files):
    
    # dictionary to keep track of playlists and tracks already added to graph
    playlist_dict = {}
    track_dict = {}
    global_edge_list = []
    
    for json_file in json_files:
        data_json = json.load(open(json_file, 'r'))
        
        # add playlist nodes
        playlist_attrs = []
        # for i, playlist in enumerate(data_json['playlists']):
        for playlist in data_json['playlists']:
            # if playlist['pid'] not in playlist_dict:
            playlist_attrs.append({'pid': playlist['pid'], 'pl_name': playlist['name'], 'pl_type': 'playlist'})
            playlist_dict[playlist['pid']] = playlist['name']
 
        for node_attrs in playlist_attrs:
            if 'pl_name' in node_attrs:
                v1 = g.add_vertex()
                for key, value in node_attrs.items():
                    g.vp[key][v1] = value
                    
        # add track nodes and edges to playlists
                               
        # for i, playlist in enumerate(data_json['playlists']):
        for playlist in data_json['playlists']:
            # print(playlist['pid'])
            playlist_edge_list=[]
            tracks_list = playlist['tracks']
            for track in tracks_list:
                pid = playlist['pid']
                if track['track_uri'] not in track_dict:
                    v2 = g.add_vertex()
                    g.vp['tr_uri'][v2] = track['track_uri']
                    g.vp['tr_name'][v2] = track['track_name']
                    g.vp['tr_type'][v2] = 'track'
                    track_dict[track['track_uri']] = v2
                else:
                    v2 = track_dict[track['track_uri']]
                # print(track_dict[track['track_uri']])
                playlist_edge_list.append((pid, track_dict[track['track_uri']]))
            playlist_edge_list = sorted(list(set(playlist_edge_list)))
            for node_pair in playlist_edge_list:
                g.add_edge(node_pair[0], node_pair[1])
            global_edge_list.append(playlist_edge_list)
                # playlist_edge_list.append((playlist['pid'], track_dict[track['track_uri']]))
            # playlist_edge_list = list(set(playlist_edge_list))
            # global_edge_list.append(playlist_edge_list)
        print(f'{json_file} processed!')
    print(f'playlist_dict length: {len(playlist_dict)}')
    print(f'track_dict length: {len(track_dict)}')
    return global_edge_list, playlist_dict, track_dict

    
    # return playlist_edge_list, playlist_dict, track_dict

In [22]:
fixed_graph = gt.Graph(directed=False)

#playlist attributes
fixed_graph.vp['pid'] = fixed_graph.new_vertex_property('string')
fixed_graph.vp['pl_name'] = fixed_graph.new_vertex_property('string')
fixed_graph.vp['pl_type'] = fixed_graph.new_vertex_property('string')
#track attributes
fixed_graph.vp['tr_uri'] = fixed_graph.new_vertex_property('string')
fixed_graph.vp['tr_name'] = fixed_graph.new_vertex_property('string')
fixed_graph.vp['tr_type'] = fixed_graph.new_vertex_property('string')
#artist attibutes
fixed_graph.vp['ar_uri'] = fixed_graph.new_vertex_property('string')
fixed_graph.vp['ar_name'] = fixed_graph.new_vertex_property('string')
fixed_graph.vp['ar_type'] = fixed_graph.new_vertex_property('string')


In [23]:
import os
folder_path = "../data_subset/"
file_paths = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_paths.append(os.path.join(root, file))

In [24]:
global_edge_list, playlist_dict, track_dict = json2graph(fixed_graph, file_paths)

../data_subset/mpd.slice.0-999.json processed!
../data_subset/mpd.slice.1000-1999.json processed!
../data_subset/mpd.slice.2000-2999.json processed!
../data_subset/mpd.slice.3000-3999.json processed!
../data_subset/mpd.slice.4000-4999.json processed!
../data_subset/mpd.slice.5000-5999.json processed!
../data_subset/mpd.slice.6000-6999.json processed!
../data_subset/mpd.slice.7000-7999.json processed!
../data_subset/mpd.slice.8000-8999.json processed!
../data_subset/mpd.slice.9000-9999.json processed!
../data_subset/mpd.slice.10000-10999.json processed!
../data_subset/mpd.slice.11000-11999.json processed!
../data_subset/mpd.slice.12000-12999.json processed!
../data_subset/mpd.slice.13000-13999.json processed!
../data_subset/mpd.slice.14000-14999.json processed!
../data_subset/mpd.slice.15000-15999.json processed!
../data_subset/mpd.slice.16000-16999.json processed!
../data_subset/mpd.slice.17000-17999.json processed!
../data_subset/mpd.slice.18000-18999.json processed!
../data_subset/mp

In [25]:
global_edge_list[1]

[(1, <Vertex object with index '1051' at 0x7fb8f24d6ac0>),
 (1, <Vertex object with index '1052' at 0x7fb8f24d6a40>),
 (1, <Vertex object with index '1053' at 0x7fb8f24d6b40>),
 (1, <Vertex object with index '1054' at 0x7fb8f24d6bc0>),
 (1, <Vertex object with index '1055' at 0x7fb8f24d6c40>),
 (1, <Vertex object with index '1056' at 0x7fb8f24d6cc0>),
 (1, <Vertex object with index '1057' at 0x7fb8f24d6d40>),
 (1, <Vertex object with index '1058' at 0x7fb8f24d6dc0>),
 (1, <Vertex object with index '1059' at 0x7fb8f24d6e40>),
 (1, <Vertex object with index '1060' at 0x7fb8f24d6f40>),
 (1, <Vertex object with index '1061' at 0x7fb8f24d6fc0>),
 (1, <Vertex object with index '1062' at 0x7fb8f24d7040>),
 (1, <Vertex object with index '1063' at 0x7fb8f24d70c0>),
 (1, <Vertex object with index '1064' at 0x7fb8f24d7140>),
 (1, <Vertex object with index '1065' at 0x7fb8f24d71c0>),
 (1, <Vertex object with index '1066' at 0x7fb8f24d7240>),
 (1, <Vertex object with index '1067' at 0x7fb8f24d72c0>

In [26]:
print("Number of vertices:", fixed_graph.num_vertices())
print("Number of edges:", fixed_graph.num_edges())

Number of vertices: 786660
Number of edges: 6656840


In [27]:
count_pl = sum(1 for v in fixed_graph.vertices() if fixed_graph.vp.pl_type[v] == "playlist")
print(f'num_playlists: {count_pl}')
count_tr = sum(1 for v in fixed_graph.vertices() if fixed_graph.vp.tr_type[v] == "track")
print(f'num_tracks: {count_tr}')


num_playlists: 101000
num_tracks: 685660


In [28]:
largest_comp = gt.extract_largest_component(fixed_graph)
g_nx = nx.Graph()

# Add edges to NetworkX graph in batches
for v in largest_comp.vertices():
    if(largest_comp.vp.pl_type[v]=='playlist'):
        typ='playlist'
    elif(largest_comp.vp.tr_type[v] == "track"):
        typ='track'
    node_attributes = {'name':largest_comp.vp.pl_name[v], 'uri':largest_comp.vp.tr_uri[v], 'type':typ}
    g_nx.add_node(int(v), **node_attributes)
for e in largest_comp.edges():
    g_nx.add_edge(int(e.source()), int(e.target()))
    
mapping = {old_label: new_label for new_label, old_label in enumerate(sorted(g_nx.nodes()))}

# reindex the nodes in the graph
g_nx = nx.relabel_nodes(g_nx, mapping)

In [29]:
print("Number of vertices:", largest_comp.num_vertices())
print("Number of edges:", largest_comp.num_edges())

Number of vertices: 690613
Number of edges: 6656794


In [30]:
count_pl = sum(1 for v in largest_comp.vertices() if fixed_graph.vp.pl_type[v] == "playlist")
print(f'num_playlists: {count_pl}')
count_tr = sum(1 for v in largest_comp.vertices() if fixed_graph.vp.tr_type[v] == "track")
print(f'num_tracks: {count_tr}')

num_playlists: 4999
num_tracks: 685614


In [31]:
print("Number of vertices:", g_nx.number_of_nodes())
print("Number of edges:", g_nx.number_of_edges())

Number of vertices: 690613
Number of edges: 6655333


In [32]:
#check for duplicate track nodes

# create a dictionary to store the attribute values
attr_dict = {}
for node in g_nx.nodes():
    attr = g_nx.nodes[node]['uri']
    if attr in attr_dict:
        attr_dict[attr].append(node)
    else:
        attr_dict[attr] = [node]
        
del attr_dict['']

keys_with_duplicates = [key for key, values in attr_dict.items() if len(values) > 1]

# print the keys with duplicates
print(keys_with_duplicates)

[]


In [33]:
num_playlists = len([n for n, d in g_nx.nodes(data=True) if d.get('type') == 'playlist'])
print(f'num_playlists: {num_playlists}')
num_tracks = len([n for n, d in g_nx.nodes(data=True) if d.get('type') == 'track'])
print(f'num_tracks: {num_tracks}')

num_playlists: 4999
num_tracks: 685614


In [34]:
filename = 'fixed_graph.pickle'
with open(filename, 'wb') as f:
    pickle.dump(g_nx, f)